In [ ]:
import numpy as np
import chromadb
import os
from groq import Groq
from dotenv import load_dotenv

load_dotenv()

In [ ]:
class RAG:
    def __init__(self, collection_name="aus_food_nutrition"):
        self.chroma_client = chromadb.CloudClient(
            tenant='a0123436-2e87-4752-8983-73168aafe2e9',
            database='nutribot',
            api_key=os.environ.get("CHROMA_API_KEY"),
        )
        self.collection = self.chroma_client.get_or_create_collection(name=collection_name)
        self.count = self.collection.count()


    def add_documents(self, docs):
        for doc in docs:
            _id = f"id{self.count}"
            self.collection.upsert(ids=[_id], documents=[doc])
            self.count += 1

    def retrieve(self, prompt, n_results=2):
        res = self.collection.query(query_texts=[prompt], n_results=n_results)
        return res.get("documents", [[]])[0]

In [19]:
rag = RAG()
rag.add_documents([
    "Vegemite is a popular Australian spread made from brewers' yeast extract.",
    "Kangaroo meat is a lean source of protein, low in fat.",
    "The Australian Dietary Guidelines recommend eating a variety of foods from the five food groups.",
    "Wattleseed, rich in protein and fiber, is a traditional Australian food, often used in baking.",
    "The Australian Heart Foundation recommends regular consumption of seafood for its omega-3 fatty acids.",
    "The Australian Dietary Guidelines advise reducing the intake of processed foods and sugary drinks.",
    "Tim Tams, an iconic Australian snack, are high in sugar and fat, making them a treat rather than a daily food.",
    "Traditional Aboriginal diets, such as bush tucker, rely on native plants, fruits, and animals.",
    "Recent studies show a rise in plant-based eating in Australia, particularly in urban areas.",
    "Australian seafood, such as barramundi and prawns, provides a lean source of protein and omega-3 fatty acids.",
    "Macadamia nuts, native to Australia, are rich in healthy monounsaturated fats.",
    "The Australian Government promotes the consumption of at least five serves of vegetables daily.",
    "Indigenous Australians have a rich history of using native plants like bush tomatoes and quandong.",
    "The average Australian consumes over 120 kilograms of meat per year, with beef being the most popular.",
    "Australian avocados are a great source of potassium and heart-healthy fats.",
    "Vegemite is often spread on toast with butter and is a common breakfast item in Australia.",
    "Kangaroo meat is considered a sustainable protein choice as kangaroos have a lower environmental impact than cattle.",
    "Bush tomatoes, used by Indigenous Australians, are high in antioxidants and are often ground into a paste.",
    "In Australia, health experts recommend including a variety of fruits and vegetables for a balanced diet.",
    "The Australian Health Survey reported that 95% of Australians do not consume enough vegetables.",
    "Studies show that the consumption of whole grains, such as oats, is linked to lower cholesterol levels.",
    "The Australian food pyramid encourages consuming dairy, lean meats, and plant-based foods in moderation.",
    "Seafood such as barramundi is often recommended for its low mercury content and high omega-3 fatty acids.",
    "Tim Tams, a chocolate biscuit, are a favorite Australian treat but are often considered a high-calorie indulgence.",
    "Pavlova, a meringue-based dessert, is a beloved dish in Australia and New Zealand.",
    "The average Australian diet contains too much added sugar, especially from soft drinks and processed foods.",
    "Damper is a traditional Australian bread, typically cooked in a campfire or oven.",
    "A balanced diet, as recommended by Australian Dietary Guidelines, includes moderate portions of protein and whole grains.",
    "Indigenous Australian food sources include kangaroo, emu, and a wide variety of native fruits and tubers.",
    "Australia’s love for seafood includes dishes like fish and chips, often served with a side of tartare sauce.",
    "Australian wines, particularly those from regions like the Barossa Valley, are recognized globally for their quality.",
    "Salmon is a popular fish in Australia, providing a rich source of vitamin D and omega-3s.",
    "The introduction of fast food chains in Australia has significantly altered the eating habits of the population.",
    "The Australian Bureau of Statistics reports that 10% of Australians are considered obese, with high sugar intake being a contributing factor.",
    "The Australian government’s health campaigns often emphasize reducing salt intake to improve heart health.",
    "Almonds are a popular snack in Australia, valued for their high vitamin E and fiber content.",
    "The rise of veganism in Australia has led to more plant-based food options in restaurants and supermarkets.",
    "Australia is one of the world’s leading producers of honey, with varieties like manuka gaining international acclaim.",
    "Quinoa is becoming a popular alternative to rice in Australia, due to its high protein and fiber content.",
    "The Australian Dietary Guidelines suggest limiting intake of processed meats, which are linked to cancer risks.",
    "The average Australian adult consumes about 15 teaspoons of added sugar each day, surpassing recommended levels.",
    "A significant number of Australians suffer from food allergies, with peanuts, shellfish, and dairy being the most common.",
    "Australian muesli bars, a common snack, can be high in sugar and are often marketed as a healthy option.",
    "The rise of organic foods in Australia has been attributed to growing consumer awareness about health and sustainability.",
    "In Australia, the consumption of energy drinks has been linked to rising rates of sleep deprivation and heart issues.",
    "Australian coastal cuisine often incorporates fresh, local seafood like oysters, prawns, and scallops.",
    "Australia’s indigenous bush tucker includes unique plant-based foods like wattleseed, finger lime, and bush banana.",
    "Avocados from Australia are a rich source of healthy fats, making them a staple in many diet plans.",
    "The Australian heart health initiative encourages people to consume less red meat and more plant-based foods.",
    "Many Australians practice the Mediterranean diet, which is known for its health benefits and use of olive oil.",
    "Rice and legumes are a key part of many Australian meals, providing essential proteins and fiber.",
    "Australian supermarkets offer a variety of gluten-free products in response to the rise of gluten intolerance.",
    "Coconut oil has gained popularity in Australia as a cooking fat, valued for its high saturated fat content.",
    "Australians are increasingly interested in 'clean eating,' focusing on whole, unprocessed foods.",
    "In Australia, meal prepping has become a popular way to manage dietary goals and prevent overeating.",
    "Australia has a growing interest in superfoods, with foods like chia seeds and acai berries gaining popularity.",
    "Chia seeds, rich in omega-3s and fiber, are often used in Australian smoothies and breakfast bowls.",
    "Indigenous Australians used bush food like the Quandong fruit for its high vitamin C content.",
    "Australia has a growing demand for low-carb diets, with products like cauliflower rice gaining popularity.",
    "Australian cuisine often features fusion dishes, blending traditional flavors with influences from Europe and Asia.",
    "A growing trend in Australian cooking is the use of sustainable seafood, as part of environmental conservation efforts.",
    "Many Australians are shifting to plant-based proteins, such as lentils, chickpeas, and tofu.",
    "Research in Australia shows that plant-based diets are linked to lower rates of chronic diseases, including heart disease.",
    "The ‘Australian way of life’ includes a strong emphasis on barbecues, where meats like sausages and steaks are often grilled.",
    "Despite its popularity, the consumption of alcohol in Australia has been declining in recent years, especially among younger generations.",
    "The rise in popularity of organic and locally grown produce in Australia is linked to a shift toward sustainability.",
    "Bush foods, like wattleseed and bush tomatoes, are being increasingly recognized for their health benefits.",
    "Australia is known for its coffee culture, with the flat white being a popular choice among locals.",
    "Australia's commitment to sustainability has led to many organic farms, offering a range of fresh, pesticide-free produce.",
    "In Australia, a growing awareness of mental health has led to a focus on diet's role in brain function and emotional well-being.",
    "The consumption of dairy in Australia is a subject of debate, with some studies linking it to bone health benefits and others suggesting risks of dairy intolerance.",
    "Australia’s unique climate and diverse ecosystems allow for a wide range of food production, from tropical fruits to cold-climate vegetables.",
    "Many Australians are adopting intermittent fasting as a way to manage weight and improve metabolism.",
    "The use of spices in Australian cooking is growing, with turmeric and cumin being added to more modern dishes for their anti-inflammatory properties.",
    "Australia's diverse culture has influenced its food scene, with cuisines from Italian, Asian, and Middle Eastern cultures becoming commonplace.",
    "Australia is a leader in the development of plant-based meat alternatives, with companies like Fable Foods gaining international attention.",
    "The Australian government encourages more sustainable food production methods, including regenerative farming practices.",
    "Studies show that regular consumption of nuts, such as almonds and cashews, can reduce the risk of heart disease in Australians.",
    "Australians enjoy a wide range of exotic fruits, such as dragon fruit and lychee, which are often found in markets.",
    "Australia’s health-conscious population is increasingly concerned with food labeling, with clear nutritional information being a priority.",
    "Many Australians are now turning to local produce, such as kangaroo meat, for its low-fat and environmentally friendly attributes.",
    "The Australian food industry is making efforts to reduce food waste, with initiatives like composting and sustainable packaging becoming more common.",
    "Australians are becoming more aware of the environmental impact of food production, driving interest in sustainable eating.",
    "Macadamia nuts are not only a popular snack but also a key ingredient in many gourmet Australian recipes.",
    "Australia's aging population has led to a growing demand for nutrient-dense foods that cater to the needs of older adults.",
    "In Australia, there is an increasing interest in natural sweeteners, such as stevia and monk fruit, as alternatives to refined sugar.",
    "Australian lamb is highly regarded for its tenderness and rich flavor, making it a common feature in Sunday roasts.",
    "Superfoods like spirulina, acai berries, and turmeric are becoming popular additions to Australian smoothies and salads.",
    "Australia’s thriving dairy industry is a significant contributor to the country’s economy, with a wide range of milk and cheese products.",
    "There is a growing trend in Australia for meal kits, which allow individuals to prepare nutritious, home-cooked meals quickly.",
    "Australians are embracing food diversity, with ingredients like miso, kimchi, and sesame oil becoming staples in many kitchens.",
    "The Australian food market is seeing an increase in the availability of gluten-free, dairy-free, and other allergen-free products.",
    "Many Australians are interested in raw food diets, which focus on uncooked, unprocessed foods like fruits, vegetables, and nuts.",
    "Australia's multicultural society has led to an explosion of fusion cuisines, blending traditional ingredients with global flavors.",
    "Native Australian spices, such as wattleseed and finger lime, are finding their way into mainstream food products.",
    "The importance of hydration is emphasized in Australia, especially given the country’s hot climate and outdoor lifestyle.",
    "A growing number of Australians are incorporating fermented foods, such as kombucha and sauerkraut, into their diets for gut health.",
    "Recent studies suggest that a high intake of omega-3-rich foods, such as fish and walnuts, can reduce inflammation in the body.",
    "The Australian government has set up health programs to encourage Australians to eat a more plant-based diet to reduce environmental impact.",
    "The Australian food system faces challenges in balancing sustainability, health, and the growing demand for convenience foods."
])

In [11]:
# Retrieve by prompt
analyzed_health_condition = """
{
    "obesity_prediction": {
        "obesity_level": "Overweight_Level_II"
        "confidence": 10%
    },
    "diabetes_prediction": {
        "diabetes": true,
        "confidence": 90%
    }
"""

weekly_plan_format = """{
"suggestion": STRING
"weekly_plan": [
    {
        "week": 1,
        "target_calories_per_day": INT,
        "focus": STRING,
        "workouts": [ARRAY OF STRINGS],
        "meal_notes": STRING,
        "reminders": [ARRAY OF STRINGS]
    },
    ... (repeat for as many weeks as appropriate)
}]"""
prompt = f"""
You are a nutrition and fitness assistant.
Below is an analyzed health condition for a user, expressed in JSON: {analyzed_health_condition}
Your task: Based on the analyzed health condition and using the retrieved knowledge, generate a weekly plan strictly in this JSON format (replace INT and STRING placeholders): {weekly_plan_format} 
"""
relevant_texts = rag.retrieve(prompt, n_results=2)
prompt = prompt + f"\tUse this as context for answering: {relevant_texts}"
print("Retrieved texts:", relevant_texts)

Retrieved texts: ['The Australian Dietary Guidelines advise reducing the intake of processed foods and sugary drinks.', 'A balanced diet, as recommended by Australian Dietary Guidelines, includes moderate portions of protein and whole grains.']


In [12]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.1-8b-instant",
)

print(chat_completion.choices[0].message.content)

Based on the given health condition and the provided knowledge, I'll generate a weekly plan for the user. 

Given the user's obesity level is categorized as Overweight_Level_II with 10% confidence in obesity prediction, and the user has diabetes with 90% confidence, here's a suggested weekly plan:

```json
{
  "suggestion": "Maintain a strict diet and regular exercise to manage your diabetes and work towards healthy weight management.",
  "weekly_plan": [
    {
      "week": 1,
      "target_calories_per_day": 1700,
      "focus": "Balancing macronutrients (proteins, healthy fats, complex carbohydrates)",
      "workouts": [
        "Monday: 30 minutes of brisk walking",
        "Wednesday: Bodyweight exercises (20 reps of push-ups, squats, lunges)",
        "Friday: 20 minutes of cycling"
      ],
      "meal_notes": "Eat frequent, balanced meals with portion control. Include whole grains, lean proteins, and plenty of vegetables and fruits.",
      "reminders": [
        "Drink at lea

# GPQA Evaluation

GPQA introduced in Nov 2023
### Model Cutoff
**llama 3.3 70b**: Dec 2023

In [37]:
from datasets import load_dataset
import random
from huggingface_hub import login
login()

In [38]:
dataset = load_dataset("Idavidrein/gpqa", "gpqa_diamond")["train"]

In [57]:
import os
import random
import re
from groq import Groq
from datasets import load_dataset

class APIUnavailable(RuntimeError):
    pass

def _extract_item(example):
    # Try revised/capitalized keys first; fall back to HF schema.
    q = example.get("Question") or example.get("question")
    ca = example.get("Correct Answer") or example.get("correct_answer")
    inc = [
        example.get("Incorrect Answer 1"),
        example.get("Incorrect Answer 2"),
        example.get("Incorrect Answer 3"),
    ]
    if not any(inc) and "incorrect_answers" in example:
        inc = example["incorrect_answers"]
    inc = [x for x in (s.strip() if isinstance(s, str) else s for s in inc) if x]
    return q, ca, inc

def evaluate_gpqa(question, options, model="llama-3.1-8b-instant"):
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    opts_str = "\n".join(f"{chr(65+i)}. {opt}" for i, opt in enumerate(options))
    prompt = (
        "You are answering a multiple-choice question. Choose the correct answer "
        "from the options below:\n\n"
        f"Question: {question}\nOptions:\n{opts_str}:"
    )
    try:
        # relevant_texts = rag.retrieve(prompt, n_results=2)

        resp = client.chat.completions.create(
            messages=[
                {"role":"system","content":"You are a multiple-choice grader. Return the answer in EXACTLY this format and nothing else:\n@@ANSWER=<LETTER>@@"},
                {"role": "user", "content": f'{prompt}'}
            ],
            model=model,
        )
        ans = (resp.choices[0].message.content or "").strip()
        # print(ans)
        m = re.compile(r"@@ANSWER=([A-D])@@\s*$", flags=re.MULTILINE).search(ans.strip())
        return m.group(1).upper() if m else None
    except Exception as e:
        print(f"API error: {e}")
        raise APIUnavailable from e

def run_eval(num_questions):
    dataset = load_dataset("Idavidrein/gpqa", "gpqa_diamond")["train"]
    correct = 0
    attempted = 0

    for i in range(min(num_questions, len(dataset))):
        q, ca, inc = _extract_item(dataset[i])
        if not (q and ca and len(inc) >= 3):
            continue

        options = inc[:3] + [ca]
        random.shuffle(options)
        true = chr(65 + options.index(ca))

        try:
            pred = evaluate_gpqa(q, options)
        except APIUnavailable:
            break

        attempted += 1
        if pred == true:
            correct += 1

        print(f"Q{i+1}: {'✓' if pred == true else '✗'}  Pred={pred or '-'}  True={true}")

    if attempted == 0:
        print("No questions evaluated.")
    else:
        pct = 100.0 * correct / attempted
        print(f"\nFinal Score: {correct}/{attempted} ({pct:.1f}%)")

if __name__ == "__main__":
    run_eval(100)

Q1: ✗  Pred=B  True=D
Q2: ✗  Pred=D  True=C
Q3: ✗  Pred=B  True=C
Q4: ✗  Pred=A  True=D
Q5: ✗  Pred=D  True=C
Q6: ✓  Pred=D  True=D
Q7: ✗  Pred=B  True=A
Q8: ✗  Pred=A  True=C
Q9: ✓  Pred=B  True=B
Q10: ✗  Pred=D  True=C
Q11: ✗  Pred=C  True=A
Q12: ✗  Pred=B  True=A
Q13: ✗  Pred=A  True=D
Q14: ✗  Pred=D  True=C
Q15: ✗  Pred=B  True=D
Q16: ✓  Pred=B  True=B
Q17: ✗  Pred=C  True=B
Q18: ✗  Pred=C  True=A
Q19: ✗  Pred=C  True=D
Q20: ✓  Pred=C  True=C
Q21: ✗  Pred=D  True=B
Q22: ✗  Pred=B  True=D
Q23: ✗  Pred=B  True=A
Q24: ✓  Pred=D  True=D
Q25: ✗  Pred=C  True=A
Q26: ✗  Pred=B  True=C
Q27: ✗  Pred=B  True=D
Q28: ✗  Pred=D  True=C
Q29: ✓  Pred=D  True=D
Q30: ✗  Pred=B  True=D
Q31: ✓  Pred=C  True=C
Q32: ✗  Pred=B  True=C
Q33: ✓  Pred=A  True=A
Q34: ✗  Pred=C  True=D
Q35: ✓  Pred=D  True=D
Q36: ✗  Pred=B  True=D
Q37: ✗  Pred=D  True=A
Q38: ✗  Pred=C  True=D
Q39: ✓  Pred=A  True=A
Q40: ✗  Pred=B  True=D
Q41: ✗  Pred=B  True=C
Q42: ✓  Pred=B  True=B
Q43: ✗  Pred=C  True=A
Q44: ✓  Pred=D  True